In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.metrics import Precision, Recall


In [2]:
# Define the base directory and paths for training and testing
base_dir = r'C:\Users\ASUS\OneDrive\Desktop\fyp2\datasets\apple_disease_classification'
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')


In [3]:
# Set up data augmentation for training with a validation split.
# Adjusted augmentation parameters to be slightly less aggressive.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,         # Reduced rotation range (was 40)
    width_shift_range=0.1,     # Reduced horizontal shift (was 0.2)
    height_shift_range=0.1,    # Reduced vertical shift (was 0.2)
    shear_range=0.1,           # Reduced shear (was 0.2)
    zoom_range=0.1,            # Reduced zoom (was 0.2)
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2       # Reserve 20% of training data for validation
)

# Data generator for test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),    # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Create validation data generator
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 1280 images belonging to 4 classes.
Found 319 images belonging to 4 classes.
Found 600 images belonging to 4 classes.


In [4]:
# Define callbacks
# Early stopping to halt training if the model stops improving on validation data.
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

# Model checkpoint to save the best model based on validation accuracy.
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)


In [5]:
# ✅ Custom Callback to Store Learning Rate
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.learning_rate)

# ✅ Add the Callback to Your Callbacks List
lr_tracker = LearningRateTracker()


In [6]:
# Build the custom CNN model
model = Sequential([
    Input(shape=(150, 150, 3)),

    # Convolutional Block 1 with L2 regularization
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.15),

    # Convolutional Block 2 with L2 regularization
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Convolutional Block 3 with L2 regularization
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.35),

    # Flatten and Fully Connected Layers with L2 regularization
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='softmax')
])


# ✅ Compile the Model with Additional Metrics
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# Display the model architecture
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 148, 148, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 72, 72, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 34, 34, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,038,660 (72.63 MB)

 Trainable params: 19,037,188 (72.62 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [7]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint, reduce_lr, lr_tracker] 
)

C:\Users\ASUS\OneDrive\Desktop\fyp2\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\ASUS\OneDrive\Desktop\fyp2\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5401 - loss: 3.4597 - precision: 0.5525 - recall: 0.5175  

C:\Users\ASUS\OneDrive\Desktop\fyp2\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.25078, saving model to best_model.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.5415 - loss: 3.4498 - precision: 0.5539 - recall: 0.5189 - val_accuracy: 0.2508 - val_loss: 9.1182 - val_precision: 0.2508 - val_recall: 0.2508 - learning_rate: 0.0010 - lr: 0.0010
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6361 - loss: 2.8911 - precision: 0.6646 - recall: 0.6188  
Epoch 2: val_accuracy did not improve from 0.25078
40/40 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.6361 - loss: 2.8906 - precision: 0.6645 - recall: 0.6187 - val_accuracy: 0.2508 - val_loss: 4.3683 - val_precision: 0.2508 - val_recall: 0.2508 - learning_rate: 0.0010 - lr: 0.0010
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6993 - loss: 2.7198 - precision: 0.7180 - recall: 0.6642  
Epoch 3: val_accuracy did not improve from 0.25078
40/40 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.6986 - loss: 2.7219 - precision: 0.7173 - re

In [49]:
import matplotlib.pyplot as plt
import numpy as np

# ✅ Extract Metrics from Training History
history_dict = history.history
epochs = range(1, len(history_dict['loss']) + 1)

# ✅ Calculate Error Rate
train_error_rate = [1 - acc for acc in history_dict['accuracy']]
val_error_rate = [1 - acc for acc in history_dict['val_accuracy']]

# ✅ Create Subplots
fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 3 Rows, 2 Columns
fig.suptitle("Training Performance Metrics", fontsize=14)

# 🎯 Plot 1: Training & Validation Accuracy
axes[0, 0].plot(epochs, history_dict['accuracy'], 'b-', label='Test Accuracy')
axes[0, 0].plot(epochs, history_dict['val_accuracy'], 'r-', label='Training and Validation Accuracy')
axes[0, 0].set_title("Test Accuracy")
axes[0, 0].legend()

# 📉 Plot 2: Training & Validation Loss
axes[0, 1].plot(epochs, history_dict['loss'], 'b-', label='Training Loss')
axes[0, 1].plot(epochs, history_dict['val_loss'], 'r-', label='Validation Loss')
axes[0, 1].set_title("Test Loss")
axes[0, 1].legend()

# 🎯 Plot 3: Precision
if 'precision' in history_dict:
    axes[1, 0].plot(epochs, history_dict['precision'], 'b-', label='Test Precision')
    axes[1, 0].plot(epochs, history_dict['val_precision'], 'r-', label='Training and Validation Precision')
    axes[1, 0].set_title("Test Precision")
    axes[1, 0].legend()

# 🎯 Plot 4: Recall
if 'recall' in history_dict:
    axes[1, 1].plot(epochs, history_dict['recall'], 'b-', label='Test Recall')
    axes[1, 1].plot(epochs, history_dict['val_recall'], 'r-', label='Training and Validation Recall')
    axes[1, 1].set_title("Test Recall")
    axes[1, 1].legend()

# 🎯 Plot 5: Error Rate
axes[2, 0].plot(epochs, train_error_rate, 'b-', label='Test Error Rate')
axes[2, 0].plot(epochs, val_error_rate, 'r-', label='Training and Validation Error Rate')
axes[2, 0].set_title("Test Error Rate")
axes[2, 0].legend()

# 🎯 Plot 6: Learning Rate (If ReduceLROnPlateau Used)
if 'lr' in history_dict:
    axes[2, 1].plot(epochs, history_dict['lr'], 'b-', label='Learning Rate')
    axes[2, 1].set_title("Learning Rate")
    axes[2, 1].legend()

# ✅ Adjust Layout & Show Plots
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


NameError: name 'history' is not defined

In [25]:
import numpy as np

# Extract history data
train_acc = np.array(history.history['accuracy']) * 100
val_acc = np.array(history.history['val_accuracy']) * 100
train_loss = np.array(history.history['loss'])
val_loss = np.array(history.history['val_loss'])
train_precision = np.array(history.history.get('precision', [])) * 100
val_precision = np.array(history.history.get('val_precision', [])) * 100
train_recall = np.array(history.history.get('recall', [])) * 100
val_recall = np.array(history.history.get('val_recall', [])) * 100
learning_rates = np.array(history.history.get('lr', []))

# Function to display min, max, and average
def display_stats(metric_name, train_values, val_values=None, is_percentage=False):
    scale = 1 if not is_percentage else 100
    unit = "%" if is_percentage else ""
    
    print(f"\n📊 {metric_name} Statistics:")
    print(f"🔹 Training - Max: {np.max(train_values):.2f}{unit}, Min: {np.min(train_values):.2f}{unit}, Avg: {np.mean(train_values):.2f}{unit}")
    
    if val_values is not None and len(val_values) > 0:
        print(f"🔹 Validation - Max: {np.max(val_values):.2f}{unit}, Min: {np.min(val_values):.2f}{unit}, Avg: {np.mean(val_values):.2f}{unit}")

# Display statistics for each metric
display_stats("Accuracy", train_acc, val_acc, is_percentage=True)
display_stats("Loss", train_loss, val_loss)

if len(train_precision) > 0:
    display_stats("Precision", train_precision, val_precision, is_percentage=True)

if len(train_recall) > 0:
    display_stats("Recall", train_recall, val_recall, is_percentage=True)

if len(learning_rates) > 0:
    print(f"\n📉 Learning Rate - Max: {np.max(learning_rates):.8f}, Min: {np.min(learning_rates):.8f}, Avg: {np.mean(learning_rates):.8f}")



📊 Accuracy Statistics:
🔹 Training - Max: 97.34%, Min: 59.69%, Avg: 87.15%
🔹 Validation - Max: 88.40%, Min: 25.08%, Avg: 72.69%

📊 Loss Statistics:
🔹 Training - Max: 3.05, Min: 0.52, Avg: 1.11
🔹 Validation - Max: 9.12, Min: 0.80, Avg: 1.65

📊 Precision Statistics:
🔹 Training - Max: 97.63%, Min: 61.17%, Avg: 88.40%
🔹 Validation - Max: 89.90%, Min: 25.08%, Avg: 73.96%

📊 Recall Statistics:
🔹 Training - Max: 96.88%, Min: 57.34%, Avg: 85.70%
🔹 Validation - Max: 86.83%, Min: 25.08%, Avg: 70.98%

📉 Learning Rate - Max: 0.00100000, Min: 0.00000195, Avg: 0.00039711


In [29]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2

# ✅ Load the Best Model
model_path = "best_model.keras"  # Update this if needed
model = load_model(model_path)

# ✅ Define Class Labels (Make sure they match the training order)
class_labels = ['Blotch_Apple', 'Normal_Apple', 'Rot_Apple', 'Scab_Apple']


In [30]:
def preprocess_image(image_path):
    """ Load and preprocess image for model prediction. """
    img = cv2.imread(image_path)  # Load Image
    img = cv2.resize(img, (150, 150))  # Resize to match training size
    img = img / 255.0  # Normalize Pixel Values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img


In [31]:
def predict_image(image_path):
    """ Predict class of an image. """
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    
    predicted_class = np.argmax(prediction, axis=1)[0]  # Get highest probability index
    confidence = np.max(prediction) * 100  # Convert to percentage
    
    print(f"🖼️ Predicted Class: {class_labels[predicted_class]} ({confidence:.2f}%)")
    return class_labels[predicted_class], confidence


In [35]:
image_path = r"C:\Users\ASUS\OneDrive\Desktop\fyp2\datasets\apple_disease_classification\Test\Normal_Apple\AnyConv.com__images (25).jpg"# Change to your image path
predicted_class, confidence = predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
🖼️ Predicted Class: Scab_Apple (94.47%)


In [39]:
# Print model summary with detailed output shapes
model.summary()

# Correct way to get layer names and shapes dynamically
for layer in model.layers:
    try:
        print(f"Layer Name: {layer.name}, Output Shape: {layer.output.shape}")
    except AttributeError:
        print(f"Layer Name: {layer.name}, Output Shape: Not Available (e.g., input layer)")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 148, 148, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 72, 72, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 34, 34, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57,113,038 (217.87 MB)

 Trainable params: 19,037,188 (72.62 MB)

 Non-trainable params: 1,472 (5.75 KB)

 Optimizer params: 38,074,378 (145.24 MB)

Layer Name: conv2d, Output Shape: (None, 148, 148, 32)
Layer Name: batch_normalization, Output Shape: (None, 148, 148, 32)
Layer Name: max_pooling2d, Output Shape: (None, 74, 74, 32)
Layer Name: dropout, Output Shape: (None, 74, 74, 32)
Layer Name: conv2d_1, Output Shape: (None, 72, 72, 64)
Layer Name: batch_normalization_1, Output Shape: (None, 72, 72, 64)
Layer Name: max_pooling2d_1, Output Shape: (None, 36, 36, 64)
Layer Name: dropout_1, Output Shape: (None, 36, 36, 64)
Layer Name: conv2d_2, Output Shape: (None, 34, 34, 128)
Layer Name: batch_normalization_2, Output Shape: (None, 34, 34, 128)
Layer Name: max_pooling2d_2, Output Shape: (None, 17, 17, 128)
Layer Name: dropout_2, Output Shape: (None, 17, 17, 128)
Layer Name: flatten, Output Shape: (None, 36992)
Layer Name: dense, Output Shape: (None, 512)
Layer Name: batch_normalization_3, Output Shape: (None, 512)
Layer Name: dropout_3, Output Shape: (None, 512)
Layer Name: dense_1, Output Shape: (None, 4)
